In [71]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

### https://www.kaggle.com/datasets/primaryobjects/voicegender

In [72]:
data = pd.read_csv('/kaggle/input/voicegender/voice.csv')

In [73]:
data

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,0.131884,0.084734,0.153707,0.049285,0.201144,0.151859,1.762129,6.630383,0.962934,0.763182,...,0.131884,0.182790,0.083770,0.262295,0.832899,0.007812,4.210938,4.203125,0.161929,female
3164,0.116221,0.089221,0.076758,0.042718,0.204911,0.162193,0.693730,2.503954,0.960716,0.709570,...,0.116221,0.188980,0.034409,0.275862,0.909856,0.039062,3.679688,3.640625,0.277897,female
3165,0.142056,0.095798,0.183731,0.033424,0.224360,0.190936,1.876502,6.604509,0.946854,0.654196,...,0.142056,0.209918,0.039506,0.275862,0.494271,0.007812,2.937500,2.929688,0.194759,female
3166,0.143659,0.090628,0.184976,0.043508,0.219943,0.176435,1.591065,5.388298,0.950436,0.675470,...,0.143659,0.172375,0.034483,0.250000,0.791360,0.007812,3.593750,3.585938,0.311002,female


In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   meanfreq  3168 non-null   float64
 1   sd        3168 non-null   float64
 2   median    3168 non-null   float64
 3   Q25       3168 non-null   float64
 4   Q75       3168 non-null   float64
 5   IQR       3168 non-null   float64
 6   skew      3168 non-null   float64
 7   kurt      3168 non-null   float64
 8   sp.ent    3168 non-null   float64
 9   sfm       3168 non-null   float64
 10  mode      3168 non-null   float64
 11  centroid  3168 non-null   float64
 12  meanfun   3168 non-null   float64
 13  minfun    3168 non-null   float64
 14  maxfun    3168 non-null   float64
 15  meandom   3168 non-null   float64
 16  mindom    3168 non-null   float64
 17  maxdom    3168 non-null   float64
 18  dfrange   3168 non-null   float64
 19  modindx   3168 non-null   float64
 20  label     3168 non-null   obje

In [75]:
y = data['label']
x = data.drop('label',axis=1).copy()
scaler = StandardScaler()
x = scaler.fit_transform(x)
y = y.apply(lambda x: 1 if x=='male' else 0)

In [76]:
y = y.apply(lambda x: 1 if x=='male' else 0)

In [77]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [78]:
x.shape

(3168, 20)

In [79]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.7,random_state=13)

In [80]:
inputs = tf.keras.Input(shape=(x.shape[1]))

x = tf.keras.layers.Dense(64,activation='relu')(inputs)
x = tf.keras.layers.Dense(128,activation='relu')(x)

outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs,outputs=outputs)

model.compile(
    optimizer='adam'
    ,loss='binary_crossentropy'
    ,metrics=['accuracy',tf.keras.metrics.AUC(name='auc')]
)

hist = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    batch_size=35,
    epochs=100,
    callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='val_loss'
        ,patience=10
        ,restore_best_weights=True)
    )

Epoch 1/100
51/51 [==============================] - 1s 9ms/step - loss: 0.1341 - accuracy: 0.9848 - auc: 0.0000e+00 - val_loss: 0.0089 - val_accuracy: 1.0000 - val_auc: 0.0000e+00
Epoch 2/100
51/51 [==============================] - 0s 3ms/step - loss: 0.0043 - accuracy: 1.0000 - auc: 0.0000e+00 - val_loss: 0.0022 - val_accuracy: 1.0000 - val_auc: 0.0000e+00
Epoch 3/100
51/51 [==============================] - 0s 3ms/step - loss: 0.0014 - accuracy: 1.0000 - auc: 0.0000e+00 - val_loss: 9.7284e-04 - val_accuracy: 1.0000 - val_auc: 0.0000e+00
Epoch 4/100
51/51 [==============================] - 0s 3ms/step - loss: 7.1406e-04 - accuracy: 1.0000 - auc: 0.0000e+00 - val_loss: 5.5868e-04 - val_accuracy: 1.0000 - val_auc: 0.0000e+00
Epoch 5/100
51/51 [==============================] - 0s 3ms/step - loss: 4.3066e-04 - accuracy: 1.0000 - auc: 0.0000e+00 - val_loss: 3.5600e-04 - val_accuracy: 1.0000 - val_auc: 0.0000e+00
Epoch 6/100
51/51 [==============================] - 0s 3ms/step - loss: 2.

In [81]:
model.evaluate(x_test,y_test)

30/30 [==============================] - 0s 2ms/step - loss: 4.1206e-07 - accuracy: 1.0000 - auc: 0.0000e+00


[4.1206240553037787e-07, 1.0, 0.0]

In [82]:
y = data['label']
x = data.drop('label',axis=1).copy()
scaler = StandardScaler()
x = scaler.fit_transform(x)
y = y.apply(lambda x: 1 if x=='male' else 0)

In [83]:
x = tf.keras.preprocessing.sequence.pad_sequences(x,dtype=np.float,maxlen=20,padding='post')

/tmp/ipykernel_32/1044518553.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = tf.keras.preprocessing.sequence.pad_sequences(x,dtype=np.float,maxlen=20,padding='post')


In [84]:
x = x.reshape(-1,4,5)

In [85]:
x = np.expand_dims(x,axis=3)

In [86]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.74,random_state = 13)

In [87]:
inputs = tf.keras.Input(shape=(4,5,1))

x = tf.keras.layers.Conv2D(16,1,activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(32,1,activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(64,1,activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(128,1,activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs,outputs=outputs)

model.compile(
    optimizer='adam'
    ,loss='binary_crossentropy'
    ,metrics=['accuracy',tf.keras.metrics.AUC(name='auc')]
)

hist = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    batch_size=35,
    epochs=100,
    callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='val_loss'
        ,patience=10
        ,restore_best_weights=True)
    )

Epoch 1/100
54/54 [==============================] - 2s 10ms/step - loss: 0.6580 - accuracy: 0.6240 - auc: 0.7306 - val_loss: 0.6212 - val_accuracy: 0.7335 - val_auc: 0.8122
Epoch 2/100
54/54 [==============================] - 0s 5ms/step - loss: 0.5944 - accuracy: 0.7285 - auc: 0.7974 - val_loss: 0.5510 - val_accuracy: 0.7719 - val_auc: 0.8293
Epoch 3/100
54/54 [==============================] - 0s 4ms/step - loss: 0.5365 - accuracy: 0.7701 - auc: 0.8306 - val_loss: 0.5058 - val_accuracy: 0.7910 - val_auc: 0.8801
Epoch 4/100
54/54 [==============================] - 0s 4ms/step - loss: 0.5073 - accuracy: 0.7909 - auc: 0.8497 - val_loss: 0.4767 - val_accuracy: 0.7932 - val_auc: 0.8827
Epoch 5/100
54/54 [==============================] - 0s 4ms/step - loss: 0.4855 - accuracy: 0.7920 - auc: 0.8613 - val_loss: 0.4536 - val_accuracy: 0.8252 - val_auc: 0.8935
Epoch 6/100
54/54 [==============================] - 0s 5ms/step - loss: 0.4606 - accuracy: 0.8107 - auc: 0.8754 - val_loss: 0.4389 - 